<a href="https://colab.research.google.com/github/SunkaraboinaPraveenKumar/Machine_Learning_Projects/blob/main/ReAct_Langchain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# %%capture
!pip install langchain>=0.1.17 openai>=1.13.3 langchain_openai>=0.1.6 transformers>=4.40.1 datasets>=2.18.0 accelerate>=0.27.2 sentence-transformers>=2.5.1 duckduckgo-search>=5.2.2 langchain_community
# !CMAKE_ARGS="-DLLAMA_CUDA=on" pip install llama-cpp-python==0.2.69

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [2]:
!wget https://huggingface.co/microsoft/Phi-3-mini-4k-instruct-gguf/resolve/main/Phi-3-mini-4k-instruct-fp16.gguf

# If this command does not work for you, you can use the link directly to download the model
# https://huggingface.co/microsoft/Phi-3-mini-4k-instruct-gguf/resolve/main/Phi-3-mini-4k-instruct-fp16.gguf

--2025-02-10 03:48:20--  https://huggingface.co/microsoft/Phi-3-mini-4k-instruct-gguf/resolve/main/Phi-3-mini-4k-instruct-fp16.gguf
Resolving huggingface.co (huggingface.co)... 3.169.137.111, 3.169.137.19, 3.169.137.5, ...
Connecting to huggingface.co (huggingface.co)|3.169.137.111|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs-us-1.hf.co/repos/41/c8/41c860f65b01de5dc4c68b00d84cead799d3e7c48e38ee749f4c6057776e2e9e/5d99003e395775659b0dde3f941d88ff378b2837a8dc3a2ea94222ab1420fad3?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27Phi-3-mini-4k-instruct-fp16.gguf%3B+filename%3D%22Phi-3-mini-4k-instruct-fp16.gguf%22%3B&Expires=1739162901&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTczOTE2MjkwMX19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy11cy0xLmhmLmNvL3JlcG9zLzQxL2M4LzQxYzg2MGY2NWIwMWRlNWRjNGM2OGIwMGQ4NGNlYWQ3OTlkM2U3YzQ4ZTM4ZWU3NDlmNGM2MDU3Nzc2ZTJlOWUvNWQ5OTAwM2UzOTU3NzU2NTliMGRkZTNmOTQxZDg4

In [9]:
from google.colab import userdata
import os
os.environ['GROQ_API_KEY'] = userdata.get('groq_api_key')

In [10]:
!pip install langchain_groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.9/121.9 kB 3.0 MB/s eta 0:00:00


In [13]:
from langchain import LlamaCpp
from langchain_groq import ChatGroq
# Make sure the model path is correct for your system!
llm = ChatGroq(
    model_name="llama-3.3-70b-versatile"
)

In [14]:
llm.invoke("Hi! My name is Maarten. What is 1 + 1?")

AIMessage(content='Nice to meet you, Maarten! The answer to 1 + 1 is 2. How can I help you with anything else today?', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 31, 'prompt_tokens': 51, 'total_tokens': 82, 'completion_time': 0.112727273, 'prompt_time': 0.009196427, 'queue_time': 0.020012157, 'total_time': 0.1219237}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_4e32347616', 'finish_reason': 'stop', 'logprobs': None}, id='run-7d4ba8eb-50a4-4ed7-8b40-d0685de06f87-0', usage_metadata={'input_tokens': 51, 'output_tokens': 31, 'total_tokens': 82})

In [15]:
from langchain import PromptTemplate

# Create a prompt template with the "input_prompt" variable
template = """<s><|user|>
{input_prompt}<|end|>
<|assistant|>"""
prompt = PromptTemplate(
    template=template,
    input_variables=["input_prompt"]
)

In [16]:
basic_chain = prompt | llm

In [17]:
# Use the chain
basic_chain.invoke(
    {
        "input_prompt": "Hi! My name is Maarten. What is 1 + 1?",
    }
)

AIMessage(content='Hi Maarten! The answer to 1 + 1 is 2. How can I help you with anything else today?', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 27, 'prompt_tokens': 66, 'total_tokens': 93, 'completion_time': 0.098181818, 'prompt_time': 0.009551091, 'queue_time': 0.039961148, 'total_time': 0.107732909}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_4196e754db', 'finish_reason': 'stop', 'logprobs': None}, id='run-81ebc758-8966-46ad-9ea3-d0611eed15f1-0', usage_metadata={'input_tokens': 66, 'output_tokens': 27, 'total_tokens': 93})

In [18]:
from langchain import LLMChain

# Create a chain for the title of our story
template = """<s><|user|>
Create a title for a story about {summary}. Only return the title.<|end|>
<|assistant|>"""
title_prompt = PromptTemplate(template=template, input_variables=["summary"])
title = LLMChain(llm=llm, prompt=title_prompt, output_key="title")

<ipython-input-18-61dd782c6da9>:8: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  title = LLMChain(llm=llm, prompt=title_prompt, output_key="title")


In [19]:
title.invoke({"summary": "a girl that lost her mother"})

{'summary': 'a girl that lost her mother', 'title': '"Beyond the Empty Chair"'}

In [20]:
# Create a chain for the character description using the summary and title
template = """<s><|user|>
Describe the main character of a story about {summary} with the title {title}. Use only two sentences.<|end|>
<|assistant|>"""
character_prompt = PromptTemplate(
    template=template, input_variables=["summary", "title"]
)
character = LLMChain(llm=llm, prompt=character_prompt, output_key="character")

In [21]:
# Create a chain for the story using the summary, title, and character description
template = """<s><|user|>
Create a story about {summary} with the title {title}. The main charachter is: {character}. Only return the story and it cannot be longer than one paragraph<|end|>
<|assistant|>"""
story_prompt = PromptTemplate(
    template=template, input_variables=["summary", "title", "character"]
)
story = LLMChain(llm=llm, prompt=story_prompt, output_key="story")

In [22]:
# Combine all three components to create the full chain
llm_chain = title | character | story

In [23]:
llm_chain.invoke("a girl that lost her mother")

{'summary': 'a girl that lost her mother',
 'title': '"Beyond the Empty Chair"',
 'character': 'The main character of "Beyond the Empty Chair" is a young girl named Lily, who is struggling to cope with the loss of her mother and find her place in a world that seems to have moved on without her. With a mix of vulnerability, determination, and curiosity, Lily navigates the complexities of grief, family, and identity as she tries to fill the void left by her mother\'s absence.',
 'story': "Beyond the Empty Chair: Lily's eyes wandered to the empty chair at the dinner table, a constant reminder of her mother's absence. It had been a year since she passed away, but the pain still felt like an open wound. As she navigated the complexities of grief, family, and identity, Lily found herself struggling to cope with the loss and find her place in a world that seemed to have moved on without her. With a mix of vulnerability, determination, and curiosity, she began to explore the memories and stori

In [24]:
# Let's give the LLM our name
basic_chain.invoke({"input_prompt": "Hi! My name is Maarten. What is 1 + 1?"})

AIMessage(content='Hi Maarten! The answer to 1 + 1 is 2. How can I help you with anything else today?', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 27, 'prompt_tokens': 66, 'total_tokens': 93, 'completion_time': 0.098181818, 'prompt_time': 0.013727583, 'queue_time': 0.06260007199999999, 'total_time': 0.111909401}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_fcc3b74982', 'finish_reason': 'stop', 'logprobs': None}, id='run-95986f52-8020-461d-8296-cb9ed2f9ba16-0', usage_metadata={'input_tokens': 66, 'output_tokens': 27, 'total_tokens': 93})

In [25]:
# Next, we ask the LLM to reproduce the name
basic_chain.invoke({"input_prompt": "What is my name?"})

AIMessage(content="I don't have any information about your name. I'm a large language model, I don't have the ability to know your personal details or retain information about our previous conversations. Each time you interact with me, it's a new conversation. If you'd like to share your name with me, I'd be happy to chat with you!", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 70, 'prompt_tokens': 55, 'total_tokens': 125, 'completion_time': 0.254545455, 'prompt_time': 0.01179353, 'queue_time': 0.428281341, 'total_time': 0.266338985}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_fcc3b74982', 'finish_reason': 'stop', 'logprobs': None}, id='run-8db08775-87f9-4adf-8857-9f3bb52afb29-0', usage_metadata={'input_tokens': 55, 'output_tokens': 70, 'total_tokens': 125})

In [26]:
# Create an updated prompt template to include a chat history
template = """<s><|user|>Current conversation:{chat_history}

{input_prompt}<|end|>
<|assistant|>"""

prompt = PromptTemplate(
    template=template,
    input_variables=["input_prompt", "chat_history"]
)

In [27]:
from langchain.memory import ConversationBufferMemory

# Define the type of Memory we will use
memory = ConversationBufferMemory(memory_key="chat_history")

# Chain the LLM, Prompt, and Memory together
llm_chain = LLMChain(
    prompt=prompt,
    llm=llm,
    memory=memory
)

<ipython-input-27-9823403f20ac>:4: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(memory_key="chat_history")


In [29]:
# Generate a conversation and ask a basic question
llm_chain.invoke({"input_prompt": "Hi! My name is Praveen. What is 1 + 1?"})

{'input_prompt': 'Hi! My name is Praveen. What is 1 + 1?',
 'chat_history': 'Human: Hi! My name is Maarten. What is 1 + 1?\nAI: Hello Maarten! The answer to 1 + 1 is 2. Is there anything else I can help you with?',
 'text': 'Hello Praveen! The answer to 1 + 1 is 2. Is there anything else I can help you with?'}

In [30]:
# Does the LLM remember the name we gave it?
llm_chain.invoke({"input_prompt": "What is my name?"})

{'input_prompt': 'What is my name?',
 'chat_history': 'Human: Hi! My name is Maarten. What is 1 + 1?\nAI: Hello Maarten! The answer to 1 + 1 is 2. Is there anything else I can help you with?\nHuman: Hi! My name is Praveen. What is 1 + 1?\nAI: Hello Praveen! The answer to 1 + 1 is 2. Is there anything else I can help you with?',
 'text': 'Your name is Praveen.'}

In [31]:
from langchain.memory import ConversationBufferWindowMemory

# Retain only the last 2 conversations in memory
memory = ConversationBufferWindowMemory(k=2, memory_key="chat_history")

# Chain the LLM, Prompt, and Memory together
llm_chain = LLMChain(
    prompt=prompt,
    llm=llm,
    memory=memory
)

<ipython-input-31-046ef635f261>:4: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferWindowMemory(k=2, memory_key="chat_history")


In [32]:
# Ask two questions and generate two conversations in its memory
llm_chain.invoke({"input_prompt":"Hi! My name is Praveen and I am 33 years old. What is 1 + 1?"})
llm_chain.invoke({"input_prompt":"What is 3 + 3?"})

{'input_prompt': 'What is 3 + 3?',
 'chat_history': 'Human: Hi! My name is Praveen and I am 33 years old. What is 1 + 1?\nAI: Hello Praveen! Nice to meet you. The answer to 1 + 1 is 2. How can I assist you further today?',
 'text': 'The answer to 3 + 3 is 6. Is there anything else I can help you with, Praveen?'}

In [33]:
# Check whether it knows the name we gave it
llm_chain.invoke({"input_prompt":"What is my name?"})

{'input_prompt': 'What is my name?',
 'chat_history': 'Human: Hi! My name is Praveen and I am 33 years old. What is 1 + 1?\nAI: Hello Praveen! Nice to meet you. The answer to 1 + 1 is 2. How can I assist you further today?\nHuman: What is 3 + 3?\nAI: The answer to 3 + 3 is 6. Is there anything else I can help you with, Praveen?',
 'text': 'Your name is Praveen.'}

In [34]:
# Check whether it knows the age we gave it
llm_chain.invoke({"input_prompt":"What is my age?"})

{'input_prompt': 'What is my age?',
 'chat_history': 'Human: What is 3 + 3?\nAI: The answer to 3 + 3 is 6. Is there anything else I can help you with, Praveen?\nHuman: What is my name?\nAI: Your name is Praveen.',
 'text': "I don't have any information about your age. Our conversation just started with a math question and a mention of your name, but I don't have any details about your age. Is there anything else I can help you with?"}

In [35]:
# Create a summary prompt template
summary_prompt_template = """<s><|user|>Summarize the conversations and update with the new lines.

Current summary:
{summary}

new lines of conversation:
{new_lines}

New summary:<|end|>
<|assistant|>"""
summary_prompt = PromptTemplate(
    input_variables=["new_lines", "summary"],
    template=summary_prompt_template
)

In [36]:
from langchain.memory import ConversationSummaryMemory

# Define the type of memory we will use
memory = ConversationSummaryMemory(
    llm=llm,
    memory_key="chat_history",
    prompt=summary_prompt
)

# Chain the LLM, prompt, and memory together
llm_chain = LLMChain(
    prompt=prompt,
    llm=llm,
    memory=memory
)

<ipython-input-36-905e44fca19a>:4: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationSummaryMemory(


In [39]:
# Generate a conversation and ask for the name
llm_chain.invoke({"input_prompt": "Hi! My name is Praveen. What is 1 + 1?"})
llm_chain.invoke({"input_prompt": "What is my name?"})

{'input_prompt': 'What is my name?',
 'chat_history': "Here is the updated summary:\n\nMaarten introduced himself and asked the AI to calculate 1 + 1, to which the AI responded with the correct answer, 2, and asked how it could assist Maarten further. The AI then confirmed Maarten's name when he asked, recalling their initial introduction. Maarten then inquired about his first question, and the AI replied that it was to calculate 1 + 1. Later, a new user, Praveen, introduced himself and asked the AI to calculate 1 + 1. The AI responded with the correct answer, 2, and offered to assist Praveen further.",
 'text': 'Your name is Praveen, according to the conversation summary.'}

In [40]:
# Check whether it has summarized everything thus far
llm_chain.invoke({"input_prompt": "What was the first question I asked?"})

{'input_prompt': 'What was the first question I asked?',
 'chat_history': "Here is the updated summary:\n\nMaarten introduced himself and asked the AI to calculate 1 + 1, to which the AI responded with the correct answer, 2, and asked how it could assist Maarten further. The AI then confirmed Maarten's name when he asked, recalling their initial introduction. Maarten then inquired about his first question, and the AI replied that it was to calculate 1 + 1. Later, a new user, Praveen, introduced himself and asked the AI to calculate 1 + 1. The AI responded with the correct answer, 2, and offered to assist Praveen further. Praveen then asked the AI about his name, and the AI replied that his name is Praveen, according to the conversation summary.",
 'text': 'According to the conversation summary, your first question was to calculate 1 + 1.'}

In [41]:
# Check what the summary is thus far
memory.load_memory_variables({})

{'chat_history': "Here is the updated summary:\n\nMaarten introduced himself and asked the AI to calculate 1 + 1, to which the AI responded with the correct answer, 2, and asked how it could assist Maarten further. The AI then confirmed Maarten's name when he asked, recalling their initial introduction. Maarten then inquired about his first question, and the AI replied that it was to calculate 1 + 1. Later, a new user, Praveen, introduced himself and asked the AI to calculate 1 + 1. The AI responded with the correct answer, 2, and offered to assist Praveen further. Praveen then asked the AI about his name, and the AI replied that his name is Praveen, according to the conversation summary. Then, the user asked the AI what their first question was, and the AI responded that, according to the conversation summary, the user's first question was to calculate 1 + 1."}

In [42]:
import os
openai_llm = llm

In [43]:
# Create the ReAct template
react_template = """Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}"""

prompt = PromptTemplate(
    template=react_template,
    input_variables=["tools", "tool_names", "input", "agent_scratchpad"]
)

In [44]:
from langchain.agents import load_tools, Tool
from langchain.tools import DuckDuckGoSearchResults

# You can create the tool to pass to an agent
search = DuckDuckGoSearchResults()
search_tool = Tool(
    name="duckduck",
    description="A web search engine. Use this to as a search engine for general queries.",
    func=search.run,
)

# Prepare tools
tools = load_tools(["llm-math"], llm=openai_llm)
tools.append(search_tool)

In [45]:
from langchain.agents import AgentExecutor, create_react_agent

# Construct the ReAct agent
agent = create_react_agent(openai_llm, tools, prompt)
agent_executor = AgentExecutor(
    agent=agent, tools=tools, verbose=True, handle_parsing_errors=True
)

In [46]:
# What is the Price of a MacBook Pro?
agent_executor.invoke(
    {
        "input": "What is the current price of a MacBook Pro in USD? How much would it cost in EUR if the exchange rate is 0.85 EUR for 1 USD?"
    }
)



> Entering new AgentExecutor chain...
To find the current price of a MacBook Pro in USD and then convert it to EUR, I first need to find the current price of a MacBook Pro in USD. 

Action: duckduck
Action Input: current price of MacBook Pro in USDsnippet: A MacBook Pro cost ranges from just over $1000 to well over $3000. Determining the true cost for the MacBook Pro you need requires close examination of the model options, hardware configurations, and Apple's pricing strategies. Let's examine how Apple prices each MacBook Pro base model and default hardware configuration: MacBook Pro 13-inch, title: How Much Does a MacBook Pro Cost? - The Pricer, link: https://www.thepricer.org/how-much-does-a-macbook-pro-cost/, snippet: Apple's 2024 MacBook Pro 16-inch can be equipped with an M4 Pro or M4 Max chip. Retail prices start at $2,499 USD, but every configuration is on sale, with the best prices in this guide knocking hundred of dollars off the laptop of your choosing. You can also browse

{'input': 'What is the current price of a MacBook Pro in USD? How much would it cost in EUR if the exchange rate is 0.85 EUR for 1 USD?',
 'output': 'The current price of a MacBook Pro in USD can range from around $1000 to over $3000, depending on the model and configuration. For the base model of the 2024 MacBook Pro 16-inch, which starts at $2,499 USD, the price in EUR would be approximately 2124.15 EUR, based on an exchange rate of 0.85 EUR for 1 USD.'}